In [ ]:
import os
import glob 
import numpy as np
from scipy.interpolate import PchipInterpolator as pcip
#from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

from scipy import stats
import pandas as pd
import statistics as stat
import matplotlib.pyplot as plt
import scipy.ndimage.filters as ndif
import statsmodels.api as sm
from matplotlib import rcParams
import seaborn as sns

In [ ]:
#Initilsetup____________________________________________________________________________________________________________

os.chdir('C:\\Users\\dell\\Desktop\\Test')
path = os.getcwd()
buildingfloor_files = glob.glob(os.path.join(path, "*Floor5.csv"))
folder_name=path.rpartition('\\')[2] 

#relevant time details
year=525600
day=1440
week=int(day*7)

#Sections where both sensor power load details availabe (2019 floor5 zone1)
datastart=int(96480+2*1440)  #96480 -67 days into 2019 start of T_room data in zone 1 2019 floor5 
dataend=int(317520-720)      #317520 end of T_room data for zone 1


#Importing building and ambient raw data
timeyear=np.arange(0,525600,1)#Time in minutes for a year 
D_f = pd.read_csv(buildingfloor_files[0])
D_f['Date'] =  pd.to_datetime(D_f['Date'], infer_datetime_format=True)
Power=D_f.iloc[:,1]+D_f.iloc[:,2]+D_f.iloc[:,3]+D_f.iloc[:,4]

ambienttemperature_file = glob.glob(os.path.join(path, "Ambient values.csv"))
A_f = pd.read_csv(ambienttemperature_file[0])
A_f.iloc[:,0]=A_f.iloc[:,0]+720


In [ ]:
#functions used___________________________________________________________________________________________________________________________

#Do not use. Only useful to fit data from daily data to give by-minute data 
def fancypantsinterpolator(D_f,one,two): #D_f:data frame containing the raw daily data,column of first entry becomes 'one' and next column becomes 'two'
    j=0
    A=np.zeros(525600)
    for i in range(1440,525600,1):
        if i%720==0 and (i/720)%2!=0:
                y=[D_f.iloc[j,one],D_f.iloc[j+1,two]]
                x=[i,i+720]
                f=pcip(x, y, axis=0, extrapolate=None)
                A[i]=D_f.iloc[j,one]
        elif i%720==0 and (i/720)%2==0:
                y=[D_f.iloc[j+1,two],D_f.iloc[j+1,one]]
                x=[i,i+720]
                f=pcip(x, y, axis=0, extrapolate=None)
                A[i]=D_f.iloc[j+1,two]
                j=j+1
        elif i%720!=0:
            A[i]=f(i)    
    return A

def knn_mean(ts, n):
    out = np.copy(ts)
    for i, val in enumerate(ts):
        if np.isnan(val):
            n_by_2 = np.ceil(n/2)
            lower = np.max([0, int(i-n_by_2)])
            upper = np.min([len(ts)+1, int(i+n_by_2)])
            ts_near = np.concatenate([ts[lower:i], ts[i:upper]])
            out[i] = np.nanmean(ts_near)
    return out

def plotrawdata(xTs,yTs,xTa,yTa,xACP,yACP,x1=datastart+day,x2=datastart+3*day):
            fig, ax1 = plt.subplots()
            ax1.set_xlabel('time, minutes')
            ax1.title.set_text('Air Conditioning Data')
            ax1.set_ylabel(str('Relative Humidity, %'), color = 'black') 
            ax1.plot(xTs, yTs, color = 'black', label='Zone RH')
            ax1.plot(xTa, yTa,'--',color = 'black', linewidth=2,label='Ambient RH')
            ax1.set_xlim([x1, x2])
            ax1.set_ylim([10, 100])
            ax1.legend(loc='lower center')
            ax2 = ax1.twinx()
            ax2.set_xlim([x1, x2])
            ax2.set_ylim([0, 60])
            ax2.set_ylabel('AC load, kW', color = 'blue') 
            ax2.tick_params(axis ='y', labelcolor = 'blue')
            ax2.plot(xACP, yACP, color = 'blue', label='')
            return plt.show()

def plotdistributiondata(x,yTs,yTamb,yaxis,title,sensor,ambient,x1=0,x2=100):
            fig, ax1 = plt.subplots()
            ax1.set_xlabel(str(yaxis))
            ax1.title.set_text(str(title))
            ax1.set_ylabel('Number Frequency', color = 'black') 
            ax1.plot(x, yTs, color = 'black', label=str(sensor))
            ax1.plot(x, yTamb,'--',color = 'black', linewidth=2,label=str(ambient))
            ax1.legend(loc='best')
            ax1.set_xlim([x1, x2])
            return plt.show()


def running_mean_uniform_filter1d(x, N):
    return ndif.uniform_filter1d(x, N, mode='constant', origin=-(N//2))[:-(N-1)]


In [ ]:
#Body2_____________________________________________________________________________________________________________________________________

#Raw data plotting
#Temperature raw data
plotrawdata(timeyear,D_f.iloc[:,7],A_f.iloc[:,0],A_f.iloc[:,1],timeyear,Power)
#RH raw data
plotrawdata(timeyear,D_f.iloc[:,8],A_f.iloc[:,0],A_f.iloc[:,2],timeyear,Power)


#Pruning data
Dsubzone1=D_f.iloc[datastart:dataend,0:10]
Dsubzone1['AC_load'] = Dsubzone1.iloc[:,1]+Dsubzone1.iloc[:,2]+Dsubzone1.iloc[:,3]+Dsubzone1.iloc[:,4]
Dsubzone1=Dsubzone1.ffill()
del Dsubzone1["z1_AC1(kW)"]
del Dsubzone1["z1_AC2(kW)"]
del Dsubzone1["z1_AC3(kW)"]
del Dsubzone1["z1_AC4(kW)"]
del Dsubzone1["z1_Light(kW)"]
del Dsubzone1["z1_Plug(kW)"]
del Dsubzone1["z1_S1(lux)"]

Asubzone1=A_f.iloc[datastart:dataend,1:3]
#Nearest neighbors both sides 10 units used to get averaged point to fill in NANs
#Dsubzone1.iloc[:,-1] = knn_mean(Dsubzone1.iloc[:,-1].values, 10)



# Kernel density distributions of datasets
kernelACloads = stats.gaussian_kde(Dsubzone1.iloc[:,-1])
kernelTsens = stats.gaussian_kde(Dsubzone1.iloc[:,1])
kernelRHsens = stats.gaussian_kde(Dsubzone1.iloc[:,2])
kernelTamb = stats.gaussian_kde(Asubzone1.iloc[:,0])
kernelRHamb = stats.gaussian_kde(Asubzone1.iloc[:,1])


fun_x=np.arange(0,150,0.5)
yac=kernelACloads(fun_x)
yTsen=kernelTsens(fun_x)
yRHsens=kernelRHsens(fun_x)
yTamb=kernelTamb(fun_x)
yRHamb=kernelRHamb(fun_x)

#Temperature detail
plotdistributiondata(fun_x,yTsen,yTamb,str("Temperature, degree Celsius"),str("Temperature Settings"),str("Zone Temperature"),str("Ambient Temperature"),10,40)
#Relative Humidity detail
plotdistributiondata(fun_x,yRHsens,yRHamb,str("Relative Humidity, %"),str("Relative Humidity Settings"),str("Zone RH"),str("Ambient RH"),25,100)


#Assimilating building attributes and external attributes for training dataset
Dmain=pd.DataFrame()
pd.DataFrame(columns = ['Date','AC_Load_kW','T_sensor','RH_sensor','T_amb','RH_amb'])                      
Dmain['Date']=Dsubzone1.iloc[:,0]
Dmain['AC_Load_kW']=Dsubzone1.iloc[:,3]
Dmain['T_sensor']=Dsubzone1.iloc[:,1]
Dmain['RH_sensor']=Dsubzone1.iloc[:,2]
Dmain['T_amb']=Asubzone1.iloc[:,0]
Dmain['RH_amb']=Asubzone1.iloc[:,1]
Dmain.index = pd.to_datetime(Dmain['Date'])
del Dmain['Date']


In [ ]:
#Analysis of all Data series: Seasonality and Trends_________________________________________________
seasonality_dict = {}
decompfreq = week #Try chnaging to day but I doubt if you'll get anything meaningful
N=25000   #Smoothing window for rolling average (~1.7 days)
for ts in Dmain.columns:
    decompositions = sm.tsa.seasonal_decompose(Dmain[ts].dropna(), period=decompfreq)
    # Store the results back
    seasonality_dict[ts] = decompositions.seasonal
# Plot seasonality
rcParams["figure.figsize"] = 16, 14
pd.DataFrame(seasonality_dict)['2019-04-01 00:00:00':'2019-05-01 00:00:00'].plot(subplots=True, layout=(4, 2), linewidth=3)


trend_dict = {}
trend_dictoriginal= {}
for ts in Dmain.columns:
    decompositions = sm.tsa.seasonal_decompose(Dmain[ts].dropna().dropna(),period=decompfreq)
    # Store back the results
    trend_dict[ts] = pd.Series(decompositions.trend).rolling(window=N).mean().iloc[N-1:]
    trend_dictoriginal[ts]=decompositions.trend
#Plot Trends
pd.DataFrame(trend_dict).plot(subplots=True, layout=(4, 3), linewidth=3)
pd.DataFrame(trend_dictoriginal).plot(subplots=True, layout=(4, 3), linewidth=3)

#Normalizing data 
normalized = Dmain.div(Dmain.max()).mul(100)  # Multiply by 100 to get percentages
normalized['2019-04-01 00:00:00':'2019-05-01 00:00:00'].plot(figsize=(16, 8), title="Parameters of interest")
plt.xlabel("Date")
plt.ylabel("Normalized intensity (%)")
plt.legend(loc='lower right')

In [ ]:
#Creating correaltion matrix 
cmap = sns.diverging_palette(250, 15, s=75, l=40, n=9, center="light", as_cmap=True) # Create a custom palette
matrix = Dmain.corr(method="pearson") # Compute corr matrix
mask = np.triu(np.ones_like(matrix, dtype=bool))
fig, ax = plt.subplots(figsize=(7, 7))
sns.heatmap(matrix, mask=mask, cmap=cmap, square=True, annot=True, fmt=".2f", ax=ax)
plt.show()


#Correlation in seasonality
seasonality_dict = {ts: sm.tsa.seasonal_decompose(Dmain[ts].dropna(),period=decompfreq).seasonal for ts in Dmain.columns}
seasonality_corr = pd.DataFrame(seasonality_dict).corr() # Compute corr matrix
sns.clustermap(seasonality_corr, annot=True, square=True)
plt.show()

#Correlation in trends
trend_dict = {ts: pd.Series(sm.tsa.seasonal_decompose(Dmain[ts].dropna(),period=decompfreq).trend).rolling(window=N).mean().iloc[N-1:] for ts in Dmain.columns}
trend_dict = pd.DataFrame(trend_dict).corr()
sns.clustermap(trend_dict, annot=True, square=True)
plt.show()


#Checking for lag:Cross-corelation of time series with highest correaltion
#Raw data
Sensor=np.array(Dmain.iloc[:,1].values)
Ambient=np.array(Dmain.iloc[:,3].values)
crosscorrelation=sm.tsa.stattools.ccf(Sensor, Ambient, adjusted=False)
plt.plot(crosscorrelation)
plt.xlim([0,year/3])
plt.title('Cross Correlation between Ambient and Sensor Temperatures')
plt.xlabel('Time, min')
plt.ylabel('Correlation Factor')

"""
SensorTrend=pd.Series(sm.tsa.seasonal_decompose(Dmain.iloc[:,1].values,period=decompfreq).trend).rolling(window=N).mean().iloc[N-1:]
SensorTrend=SensorTrend.to_numpy()
AmbientTrend=pd.Series(sm.tsa.seasonal_decompose(Dmain.iloc[:,3].values,period=decompfreq).trend).rolling(window=N).mean().iloc[N-1:]
AmbientTrend=AmbientTrend.to_numpy()
crosscorrelation2=sm.tsa.stattools.ccf(SensorTrend, AmbientTrend, adjusted=False)
plt.plot(crosscorrelation2)
plt.xlim([0,year/3])
plt.title('Cross Correlation between Ambient and Sensor Temperatures')
plt.xlabel('Time, min')
plt.ylabel('Correlation Factor')
"""
"""
#Granger Causality Tests
from statsmodels.tsa.stattools import grangercausalitytests
grangercausalitytests(Dmain[['T_sensor', 'T_amb']], maxlag=2)
"""

In [ ]:
#VAR Modelling_____________________________________________________________________________
from statsmodels.tsa.api import VAR
model = VAR(Dmain)
xx = model.select_order(maxlags=12)
results_summary=xx.summary()
#Order chosen is 4
model_fitted = model.fit(4)
model_fitted.summary()

"""
from statsmodels.stats.stattools import durbin_watson #checking residuals to see how good the model is
out = durbin_watson(model_fitted.resid)
"""

In [ ]:
#Prediction data
datastart2=470237
dataend2=D_f.shape[0]
Dsubzone2=D_f.iloc[datastart2:dataend2,0:10]
Dsubzone2['AC_load'] = Dsubzone2.iloc[:,1]+Dsubzone2.iloc[:,2]+Dsubzone2.iloc[:,3]+Dsubzone2.iloc[:,4]
Dsubzone2=Dsubzone2.ffill()
del Dsubzone2["z1_AC1(kW)"]
del Dsubzone2["z1_AC2(kW)"]
del Dsubzone2["z1_AC3(kW)"]
del Dsubzone2["z1_AC4(kW)"]
del Dsubzone2["z1_Light(kW)"]
del Dsubzone2["z1_Plug(kW)"]
del Dsubzone2["z1_S1(lux)"]
Asubzone2=A_f.iloc[datastart2:dataend2,1:3]

In [ ]:
#Assimilating building attributes and external attributes for forcast/prediction dataset
Dmain2=pd.DataFrame()
pd.DataFrame(columns = ['Date','AC_Load_kW','T_sensor','RH_sensor','T_amb','RH_amb'])                      
Dmain2['Date']=Dsubzone2.iloc[:,0]
Dmain2['AC_Load_kW']=Dsubzone2.iloc[:,3]
Dmain2['T_sensor']=Dsubzone2.iloc[:,1]
Dmain2['RH_sensor']=Dsubzone2.iloc[:,2]
Dmain2['T_amb']=Asubzone2.iloc[:,0]
Dmain2['RH_amb']=Asubzone2.iloc[:,1]
Dmain2.index = pd.to_datetime(Dmain2['Date'])
del Dmain2['Date']



"""
#getting the predicted data 
def invert_transformation(D_main, D_main2, second_diff=False):
    ##Revert back the differencing to get the forecast to original scale.
    df_fc = D_main2.copy()
    columns = D_main.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (D_main[col].iloc[-1]-D_main[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = D_main[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc
"""


nobs=4
fc = model_fitted.forecast(y=Dmain2, steps=nobs)
df_forecast = pd.DataFrame(fc, index=Dmain.index[-nobs:], columns=Dmain.columns)
      
df_forecast.loc[:, ['rgnp_forecast', 'pgnp_forecast', 'ulc_forecast', 'gdfco_forecast',
                   'gdf_forecast', 'gdfim_forecast', 'gdfcf_forecast', 'gdfce_forecast']]


"""
#Plotting 
fig, axes = plt.subplots(nrows=int(len(Dmain1.columns)/2), ncols=2, dpi=150, figsize=(10,10))
for i, (col,ax) in enumerate(zip(Dmain2.columns, axes.flatten())):
    df_results[col+'_forecast'].plot(legend=True, ax=ax).autoscale(axis='x',tight=True)
    Dmain2[col][-nobs:].plot(legend=True, ax=ax);
    ax.set_title(col + ": Forecast vs Actuals")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout()
"""


"""
#Formula saving
decompfreq = week
decomposition = sm.tsa.seasonal_decompose(Dsubzone1.iloc[:,-1].values,period=decompfreq)
trend = decomposition.trend
season = decomposition.seasonal 
"""

"""
#This section contains code that obtained daily data of temperature and relative humidity to interpolate and obtain minute by minute data
#Since that data is already available as a csv file this section is commented out

ambienttemperature_file = glob.glob(os.path.join(path, "2019-Ambient-temperature-relativehumidity.csv"))
#Reading the daily ambient Temperataure and Relative humidity values (Max, Av and Min) for the year 2019
AmbientTandR_df = pd.read_csv(ambienttemperature_file[0])
#Converting temperature from fahrenheit to celsius 
AmbientTandR_df.iloc[:,1]=(AmbientTandR_df.iloc[:,1]-32)*(5/9)
AmbientTandR_df.iloc[:,2]=(AmbientTandR_df.iloc[:,2]-32)*(5/9)
AmbientTandR_df.iloc[:,3]=(AmbientTandR_df.iloc[:,3]-32)*(5/9)

#obtaining interpolated data
T_amb=fancypantsinterpolator(AmbientTandR_df,3,1)
RH_amb=fancypantsinterpolator(AmbientTandR_df,4,6)
"""